In [106]:
import pandas as pd
import numpy as np
import pickle

In [107]:
# get current strike
current_strike = 45152
def get_current_strike():
    """ Get current strike price
    """
    return current_strike


def get_atm_strike(current_strike:int) -> int :
    atm_strike =current_strike - current_strike % -50
    #print(atm_strike)
    return atm_strike



## Base Vega - Start of Processing

In [133]:
base_filename = "BANKNIFTY_2023-09-13_option_chain_2023-09-09-07-59-50.csv"

In [139]:
# Read base file
if os.path.exists("base_vega.pkl"): 
    print("Remove base vega pkl")
    os.remove("base_vega.pkl")

df = pd.read_csv(base_filename)

current_strike = get_current_strike()
atm_strike = get_atm_strike(current_strike)
print(atm_strike)

# Change column names to more friendly ones
columns = [ x.replace(' ','').lower() for x in df.columns ]
df.columns = columns

# Remove Missing records
df = df[df.putgamma != '--']
df = df[df.callgamma != '--']
df = df[df.pcr != '--']
df = df[df['putpop%'] != '--']

#Change column datatypes
float_columns = ['putgamma', 'putvega', 'puttheta', 'putdelta', 'pcr', 'putpop%',
        'putchange%',  'putbidprice',
       'putofferprice', 'putintrinsicvalue(fut)', 'putintrinsicvalue(spot)',
       'puttimevalue', 'putltp',  'iv',  'callltp',
       'calltimevalue', 'callintrinsicvalue(spot)', 'callintrinsicvalue(fut)',
       'callbidprice', 'callofferprice',  'callchange%',
        'callpop%', 'calldelta', 'calltheta',
       'callvega', 'callgamma']

for col in float_columns:
    #print(col)
    df[col] = df[col].astype(np.float64)

df['strike'] = df['strike'].astype(np.int64)

df_vega = df[['strike','putvega','callvega']]
#df_vega.head()
if not os.path.exists("base_vega.pkl"):
    print("base vega not present. Save to pkl")
    df_vega.to_pickle("base_vega.pkl")
else:
    print("base vega pkl present")
    base_df = pd.read_pickle("base_vega.pkl")

base_call_vega_df = df_vega[df_vega['strike'] >= atm_strike]
base_put_vega_df = df_vega[df_vega['strike'] <= current_strike]
entrylist = [{'strike':current_strike,'callvega':base_call_vega_df, 'putvega':base_call_vega_df}]
for entry in entrylist:
    print(entry)

Remove base vega pkl
45200
base vega not present. Save to pkl
{'strike': 45152, 'callvega':     strike  putvega  callvega
40   45200    21.08     21.08
41   45300    20.57     20.57
42   45400    19.30     19.30
43   45500    17.59     17.59
44   45600    15.36     15.36
45   45700    13.08     13.08
46   45800    10.89     10.89
47   45900     8.83      8.83
48   46000     7.27      7.27
49   46100     6.03      6.03
50   46200     4.78      4.78
51   46300     4.18      4.18
53   46500     3.07      3.07
54   46600     2.80      2.80
57   46900     1.94      1.94
58   47000     1.80      1.80
59   47100     1.46      1.46
63   47500     1.22      1.22
66   48000     0.95      0.95
71   50500     0.49      0.49, 'putvega':     strike  putvega  callvega
40   45200    21.08     21.08
41   45300    20.57     20.57
42   45400    19.30     19.30
43   45500    17.59     17.59
44   45600    15.36     15.36
45   45700    13.08     13.08
46   45800    10.89     10.89
47   45900     8.83      8

In [135]:
df_vega.putvega.

,strike,putvega,callvega
0,40500,0.96,0.96
1,41000,1.14,1.14
3,41500,1.20,1.20
7,41900,1.38,1.38
8,42000,1.42,1.42


## Read from base vega

In [118]:
if not os.path.exists("base_vega.pkl"):
    print("base vega not present. Error !!!")
else:
    print("base vega pkl present. Read from it..")
    base_df = pd.read_pickle("base_vega.pkl")

base_call_vega_df = df_vega[df_vega['strike'] >= atm_strike]
base_put_vega_df = df_vega[df_vega['strike'] <= current_strike]

base vega pkl present. Read from it..


In [119]:
base_call_vega = base_call_vega_df.callvega.sum()

In [120]:
base_put_vega = base_put_vega_df.putvega.sum() 

In [122]:
print(f"Call Vega(start): {base_call_vega}, Put Vega(Start): {base_put_vega}")

Call Vega(start): 162.69000000000003, Put Vega(Start): 218.91999999999996


### Re-run for recent vega

In [123]:
recent_filename = "BANKNIFTY_2023-09-13_option_chain_2023-09-09-07-59-50.csv"

In [127]:
# Read new file

df = pd.read_csv(recent_filename)

current_strike = get_current_strike()
atm_strike = get_atm_strike(current_strike)
print(atm_strike)

# Change column names to more friendly ones
columns = [ x.replace(' ','').lower() for x in df.columns ]
df.columns = columns

# Remove Missing records
df = df[df.putgamma != '--']
df = df[df.callgamma != '--']
df = df[df.pcr != '--']
df = df[df['putpop%'] != '--']

#Change column datatypes
float_columns = ['putgamma', 'putvega', 'puttheta', 'putdelta', 'pcr', 'putpop%',
        'putchange%',  'putbidprice',
       'putofferprice', 'putintrinsicvalue(fut)', 'putintrinsicvalue(spot)',
       'puttimevalue', 'putltp',  'iv',  'callltp',
       'calltimevalue', 'callintrinsicvalue(spot)', 'callintrinsicvalue(fut)',
       'callbidprice', 'callofferprice',  'callchange%',
        'callpop%', 'calldelta', 'calltheta',
       'callvega', 'callgamma']

for col in float_columns:
    #print(col)
    df[col] = df[col].astype(np.float64)

df['strike'] = df['strike'].astype(np.int64)

df_vega = df[['strike','putvega','callvega']]
df_vega.head()


curr_call_vega_df = df_vega[df_vega['strike'] >= atm_strike]
curr_put_vega_df = df_vega[df_vega['strike'] <= current_strike]
curr_call_vega = curr_call_vega_df.callvega.sum()
curr_put_vega = curr_put_vega_df.putvega.sum()
print(f"Call Vega(start): {base_call_vega}, Put Vega(Start): {base_put_vega}")
print(f"Call Vega(start): {curr_call_vega}, Put Vega(Start): {curr_put_vega}")


45200
Call Vega(start): 162.69000000000003, Put Vega(Start): 218.91999999999996
Call Vega(start): 162.69000000000003, Put Vega(Start): 218.91999999999996


In [131]:
print("**** Difference *****")
call_diff = base_call_vega - curr_call_vega
put_diff = base_put_vega - curr_put_vega
print(f"Call: {call_diff}, Put: {put_diff} ")

**** Difference *****
Call: 0.0, Put: 0.0 


In [ ]:
entrylist = {'strike':current_strike,'callvega':curr_call_vega, 'putvega':curr_put_vega}